In [61]:
from sqlalchemy import create_engine
from sqlalchemy import (
    Column,
    ForeignKey,
    Integer,
    DateTime,
    Float,
    DateTime,
    Float,
    String,
    Text,
)
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker
import sqlalchemy as db
from sqlalchemy.ext.declarative import declarative_base

USERNAME = "ilarima"
PASSWORD = "Aa_1234567"
SERVER = "localhost"

# DEFINE THE ENGINE (CONNECTION OBJECT)
engine = create_engine(
    f"mysql+pymysql://{USERNAME}:{PASSWORD}@{SERVER}:3306/currencies", echo=True
)

Base = declarative_base()

/tmp/ipykernel_34657/39248693.py:27: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [62]:
class Currency(Base):
    __tablename__ = "currency"

    id = Column(Integer, primary_key=True, unique=True, autoincrement=True)
    name = Column(String(255), nullable=True)
    symbol = Column(String(255), nullable=True)
    rank = Column(Integer, nullable=True)
    main_link = Column(Text, nullable=True)
    historical_link = Column(Text, nullable=True)
    circulating_supply = Column(Float, nullable=True)

    # github = relationship("Github", back_populates="currency")  # , uselist=False
    historicals = relationship("Historical", back_populates="currency")
    languages = relationship(
        "Language", secondary="languages_currency", back_populates="currencies"
    )
    tags = relationship("Tag", secondary="tags_currency", back_populates="currencies")

    def __repr__(self):
        return f"<Currency(id={self.id}, name='{self.name}', symbol='{self.symbol}')>"

In [63]:
class GitHub(Base):
    __tablename__ = "github"

    id = Column(Integer, primary_key=True, unique=True, autoincrement=True)
    currency_id = Column(Integer, ForeignKey("currency.id"), nullable=True)
    commits_count = Column(Integer, nullable=True)
    contributors_count = Column(Integer, nullable=True)
    forks_count = Column(Integer, nullable=True)
    stars_count = Column(Integer, nullable=True)
    github_link = Column(Text, nullable=True)

    # Define a foreign key relationship to the 'currency' table
    currency = relationship("Currency")

    def __repr__(self):
        return f"<GitHub(id={self.id}, currency_id={self.currency_id})>"

In [64]:
class Historical(Base):
    __tablename__ = "historical"

    id = Column(Integer, primary_key=True, unique=True, autoincrement=True)
    currency_id = Column(Integer, ForeignKey("currency.id"), nullable=True)
    timeOpen = Column(DateTime, nullable=True)
    timeClose = Column(DateTime, nullable=True)
    timeHigh = Column(DateTime, nullable=True)
    timeLow = Column(DateTime, nullable=True)
    open = Column(Float, nullable=True)
    high = Column(Float, nullable=True)
    low = Column(Float, nullable=True)
    close = Column(Float, nullable=True)
    volume = Column(Float, nullable=True)
    marketCap = Column(Float, nullable=True)
    timestamp = Column(DateTime, nullable=True)

    # Define a foreign key relationship to the 'currency' table
    currency = relationship("Currency", back_populates="historicals")

    def __repr__(self):
        return f"<Historical(id={self.id}, currency_id={self.currency_id}, timestamp={self.timestamp})>"

In [65]:
class Language(Base):
    __tablename__ = "languages"

    id = Column(Integer, primary_key=True, unique=True, nullable=False)
    name = Column(String(255), nullable=True)

    # Define a one-to-many relationship with 'tags_currency' table
    currencies = relationship(
        "Currency", secondary="languages_currency", back_populates="languages"
    )

    def __repr__(self):
        return f"<Language(id={self.id}, name='{self.name}')>"

In [66]:
class LanguageCurrency(Base):
    __tablename__ = "languages_currency"

    id = Column(Integer, primary_key=True, unique=True, autoincrement=True)
    currency_id = Column(Integer, ForeignKey("currency.id"), nullable=True)
    language_id = Column(Integer, ForeignKey("languages.id"), nullable=True)
    percentage = Column(Float, nullable=True)

    def __repr__(self):
        return f"<LanguageCurrency(id={self.id},currency_id={self.currency_id}, language_id={self.language_id}, percentage={self.percentage})>"

In [67]:
class Tag(Base):
    __tablename__ = "tags"

    id = Column(Integer, primary_key=True, nullable=False)
    name = Column(String(255), nullable=True)

    # Define a one-to-many relationship with 'tags_currency' table
    currencies = relationship(
        "Currency", secondary="tags_currency", back_populates="tags"
    )

    def __repr__(self):
        return f"<Tag(id={self.id}, name='{self.name}')>"

In [68]:
class TagsCurrency(Base):
    __tablename__ = "tags_currency"

    id = Column(Integer, primary_key=True, autoincrement=True)
    currency_id = Column(Integer, ForeignKey("currency.id"), nullable=True)
    tag_id = Column(Integer, ForeignKey("tags.id"), nullable=True)

    def __repr__(self):
        return f"<TagsCurrency(id={self.id}, currency_id={self.currency_id}, tag_id={self.tag_id})>"

In [69]:
import pandas as pd

In [70]:
# CREATE A SESSION OBJECT TO INITIATE QUERY
# IN DATABASE
Session = sessionmaker(bind=engine)
session = Session()


# SELECT currency_id, volume, marketCap FROM historical
results = (
    session.query(Historical.currency_id, Historical.volume, Historical.marketCap)
    .where(Historical.currency_id == 154)
    .all()
)

df = pd.DataFrame(
    [(r.currency_id, r.volume, r.marketCap) for r in results],
    columns=["currency_id", "volume", "marketCap"],
)

2023-09-05 19:35:56,590 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-09-05 19:35:56,592 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-05 19:35:56,599 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-09-05 19:35:56,603 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-05 19:35:56,609 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-09-05 19:35:56,611 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-05 19:35:56,617 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2023-09-05 19:35:56,711 INFO sqlalchemy.engine.Engine SELECT historical.currency_id AS historical_currency_id, historical.volume AS historical_volume, historical.`marketCap` AS `historical_marketCap` 
FROM historical 
WHERE historical.currency_id = %(currency_id_1)s
2023-09-05 19:35:56,713 INFO sqlalchemy.engine.Engine [generated in 0.00187s] {'currency_id_1': 154}


In [ ]:
from sqlalchemy import text

conn = engine.connect()

# Define the SQL query
sql_query = text(
    """
    SELECT currency_id, volume, marketCap, timestamp
    FROM historical
"""
)

# Method 2 using sql alchemy

# query = (
#     session.query(Historical.currency_id, Historical.volume, Historical.marketCap, Historical.timestamp)
# )

# results = query.all()

# Execute the SQL query
result = conn.execute(sql_query)

# Fetch the results
selected_rows = result.fetchall()

# Print the selected rows
# for row in selected_rows:
#     print(row)

# Close the database connection when you're done
conn.close()

df = pd.DataFrame(
    [(r.currency_id, r.volume, r.marketCap, r.timestamp) for r in selected_rows],
    columns=["currency_id", "volume", "marketCap","timestamp"],
)

2023-09-05 19:35:56,875 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-05 19:35:56,878 INFO sqlalchemy.engine.Engine 
    SELECT currency_id, volume, marketCap
    FROM historical

2023-09-05 19:35:56,881 INFO sqlalchemy.engine.Engine [generated in 0.00644s] {}


2023-09-05 19:35:58,971 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
print(df.shape)
df.head()

In [ ]:
from sqlalchemy import text

conn_2 = engine.connect()

# Define the SQL query
sql_query_2 = text(
    """
    SELECT id, name, symbol, circulating_supply
    FROM currency
"""
)

# Method 2 using sql alchemy

# query = (
#     session.query(Currency.id, Currency.name, Currency.symbol, Currency.circulating_supply)
# )

# results = query.all()

# Execute the SQL query
result_2 = conn_2.execute(sql_query_2)

# Fetch the results
selected_rows_2 = result_2.fetchall()

#update_historical = output.fetchall()
# Print the selected rows
# for row in selected_rows:
#     print(row)

# Close the database connection when you're done
conn_2.close()

df_2 = pd.DataFrame(
    [( r.id, r.name, r.symbol, r.circulating_supply) for r in selected_rows_2],
    columns=[ "id", "name", "symbol","circulating_supply"],
)

In [ ]:
print(df_2.shape)
df_2.head()

In [ ]:
from sqlalchemy import text

conn_3 = engine.connect()

metadata = db.MetaData()
currency = db.Table('currency', metadata, autoload=True, autoload_with=engine)
historical = db.Table('historical', metadata, autoload=True, autoload_with=engine)


# Define the SQL query
sql_query_3 = text(
    """
    SELECT c.name, c.symbol, c.circulating_supply, h.currency_id , h.volume, h.marketCap, h.open, h.close, h.timestamp
    FROM currency AS c
    JOIN historical AS h ON c.id = h.currency_id;
"""
)

# Method 2 using sql alchemy

# query = (
#     session.query(Currency.name, Currency.symbol, Currency.circulating_supply,
#                   Historical.currency_id, Historical.volume, Historical.marketCap,
#                   Historical.open, Historical.close, Historical.timestamp)
#     .join(Historical, Currency.id == Historical.currency_id)
# )

# results = query.all()

# Execute the SQL query
result_3 = conn_3.execute(sql_query_3)

# Fetch the results
selected_rows_3 = result_3.fetchall()

# Close the database connection when you're done
conn_3.close()

df_3 = pd.DataFrame(
    [( r.currency_id, r.name, r.symbol, r.circulating_supply, r.volume, r.marketCap, r.open, r.close, r.timestamp) for r in selected_rows_3],
    columns=[ "Currency_id", "Name", "Symbol","Circulating_Supply", "Volume", "Marketcap", "Open", "Close", "Date"],
)

In [ ]:
print(df_3.shape)
df_3.head()

# Question 2 : Coins Correlation

In [ ]:
import pandas as pd

# Assume df is a DataFrame with columns 'date', 'symbol' and 'Price_Change'
df_3['Price_Change'] = df_3['Close'] - df_3['Open']
df_3['Direction'] = df_3['Price_Change'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
df_pivot = df_3.pivot(index='Date', columns='Symbol', values='Direction')

# Count the number of days two currencies change in the same direction
same_direction_days = df_pivot.corr().stack().rename_axis(('Symbol1', 'Symbol2')).reset_index(name='Correlation')
same_direction_days.columns = ['Symbol1', 'Symbol2', 'Correlation']
same_direction_days = same_direction_days[same_direction_days['Symbol1'] < same_direction_days['Symbol2']]
same_direction_days['same_direction_days'] = same_direction_days['Correlation'] * len(df_pivot)
same_direction_days['same_direction_days'] = same_direction_days['same_direction_days'].round().astype(int)
same_direction_days = same_direction_days.sort_values('same_direction_days', ascending=False)

# Print the top 30 pairs
print(same_direction_days.head(30))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Get the top 30 pairs
top_30 = same_direction_days.sort_values('same_direction_days', ascending=False).head(30)

# Filter the correlation matrix for the top symbols
symbols = list(set(top_30['Symbol1'].tolist() + top_30['Symbol2'].tolist()))

# Create the correlation matrix
corr_subset = df_pivot[symbols].corr()

# Create the figure and the axes
fig, ax = plt.subplots(figsize=(12, 10))
# Plot the heatmap with the filtered correlation matrix

sns.heatmap(corr_subset, annot=False, cmap='viridis', ax=ax)
ax.set_title('Top 30 Currency Correlation Heatmap')
plt.show()

# Qestion 5 : Top 10 Coins in Red Days

In [ ]:
import matplotlib.pyplot as plt

# Assume df is a DataFrame with columns 'date', 'symbol' and 'Price_Change'
df_red_days = df_3[df_3['Price_Change'] < 0]

# Calculate the total volume for each symbol in the red days
total_volume = df_red_days.groupby('Symbol')['Volume'].sum().sort_values(ascending=False)

# Select the top 10 symbols
top_10_symbols = total_volume.head(10)

# Plot the bar chart
plt.bar(top_10_symbols.index, top_10_symbols.values)
plt.xlabel('Symbol')
plt.ylabel('Total Volume')
plt.title('Total Volume of Top 10 Cryptocurrencies in Red Days')
plt.show()